In [7]:
# Coffee Chat Booking Agent - Real Google APIs Implementation
# !pip install cohere google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

import cohere
import os
import json
from datetime import datetime, timedelta, timezone
from typing import Dict, List, Any
import base64
from email.mime.text import MIMEText
import pytz

# Google API imports
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# OAuth 2.0 scopes
SCOPES = [
    'https://www.googleapis.com/auth/calendar',
    'https://www.googleapis.com/auth/gmail.send'
]

class GoogleAPIManager:
    def __init__(self, credentials_file='credentials.json', token_file='token.json'):
        """
        Initialize Google API manager with OAuth2 authentication
        
        Setup instructions:
        1. Go to Google Cloud Console
        2. Create a new project or select existing
        3. Enable Calendar API and Gmail API
        4. Create OAuth 2.0 credentials (Desktop application)
        5. Download the credentials.json file
        """
        self.credentials_file = credentials_file
        self.token_file = token_file
        self.creds = self._authenticate()
        self.calendar_service = build('calendar', 'v3', credentials=self.creds)
        self.gmail_service = build('gmail', 'v1', credentials=self.creds)

    def _authenticate(self):
        """Handle OAuth2 authentication flow"""
        creds = None
        
        # Load existing token
        if os.path.exists(self.token_file):
            creds = Credentials.from_authorized_user_file(self.token_file, SCOPES)
        
        # If there are no (valid) credentials available, let the user log in
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                if not os.path.exists(self.credentials_file):
                    raise FileNotFoundError(
                        f"Please download credentials.json from Google Cloud Console and place it in the current directory"
                    )
                flow = InstalledAppFlow.from_client_secrets_file(self.credentials_file, SCOPES)
                creds = flow.run_local_server(port=0)
            
            # Save the credentials for the next run
            with open(self.token_file, 'w') as token:
                token.write(creds.to_json())
        
        return creds

class CoffeeChatAgent:
    def __init__(self, cohere_api_key: str):
        self.co = cohere.Client(api_key=cohere_api_key)
        self.google_api = GoogleAPIManager()
        self.conversation_history = []
        self.tools = self._setup_tools()
        
    def _setup_tools(self):
        """Define the tools available to the agent"""
        return [
            {
                "name": "check_calendar_availability",
                "description": "Check Vach's calendar availability for the specified date range",
                "parameter_definitions": {
                    "date_range": {
                        "description": "Date range to check (e.g., 'next week', 'this Friday', 'January 15-20')",
                        "type": "str",
                        "required": True
                    },
                    "duration": {
                        "description": "Meeting duration in minutes (default: 30)",
                        "type": "int",
                        "required": False
                    }
                }
            },
            {
                "name": "create_google_meet_event",
                "description": "Create a Google Calendar event with Google Meet link",
                "parameter_definitions": {
                    "date": {
                        "description": "Meeting date in YYYY-MM-DD format",
                        "type": "str",
                        "required": True
                    },
                    "time": {
                        "description": "Meeting time in HH:MM format (24-hour)",
                        "type": "str",
                        "required": True
                    },
                    "duration": {
                        "description": "Meeting duration in minutes",
                        "type": "int",
                        "required": True
                    },
                    "attendee_email": {
                        "description": "Email address of the person booking the meeting",
                        "type": "str",
                        "required": True
                    },
                    "topic": {
                        "description": "Meeting topic or purpose",
                        "type": "str",
                        "required": False
                    }
                }
            },
            {
                "name": "send_confirmation_email",
                "description": "Send confirmation email to the attendee via Gmail",
                "parameter_definitions": {
                    "attendee_email": {
                        "description": "Email address of the attendee",
                        "type": "str",
                        "required": True
                    },
                    "event_details": {
                        "description": "JSON string containing meeting details",
                        "type": "str", 
                        "required": True
                    }
                }
            }
        ]

    def _parse_date_range(self, date_range: str) -> tuple:
        """Parse natural language date range into datetime objects"""
        now = datetime.now()
        
        if "next week" in date_range.lower():
            start = now + timedelta(days=(7 - now.weekday()))
            end = start + timedelta(days=7)
        elif "this week" in date_range.lower():
            start = now
            end = now + timedelta(days=(6 - now.weekday()))
        elif "tomorrow" in date_range.lower():
            start = now + timedelta(days=1)
            end = start + timedelta(days=1)
        else:
            # Default to next 7 days
            start = now
            end = now + timedelta(days=7)
        
        return start, end

    def list_available_calendars(self):
        """List all available calendars"""
        try:
            calendars_result = self.google_api.calendar_service.calendarList().list().execute()
            calendars = calendars_result.get('items', [])
            
            print("Available calendars:")
            for calendar in calendars:
                print(f"  - {calendar['summary']} (ID: {calendar['id']})")
                if calendar.get('primary'):
                    print(f"    ^ This is your PRIMARY calendar")
            
            return calendars
        except Exception as e:
            print(f"Error listing calendars: {e}")
            return []

        # Add this method to your agent class

    def check_calendar_availability(self, date_range: str, duration: int = 30) -> Dict:
        """Check real calendar availability using Google Calendar API"""
        try:
            print(f"🗓️  Checking calendar availability for: {date_range}")
            
            start_time, end_time = self._parse_date_range(date_range)
            
            # Query for busy times
            body = {
                "timeMin": start_time.isoformat() + 'Z',
                "timeMax": end_time.isoformat() + 'Z',
                "items": [{"id": "primary"}]
            }
            
            eventsResult = self.google_api.calendar_service.freebusy().query(body=body).execute()
            busy_times = eventsResult['calendars']['primary']['busy']
            
            # Generate available slots (9 AM to 5 PM, excluding busy times)
            available_slots = []
            current_date = start_time.date()
            end_date = end_time.date()
            
            # Use Eastern timezone
            eastern = pytz.timezone('America/New_York')

            min_advance_hours = 24
            earliest_booking_time = datetime.now(eastern) + timedelta(hours=min_advance_hours)
    
            
            while current_date <= end_date:
                # Skip weekends
                if current_date.weekday() < 5:  # Monday = 0, Sunday = 6
                    # Check 9 AM to 5 PM in 30-minute intervals
                    for hour in range(9, 21):
                        for minute in [0, 30]:
                            # Create timezone-aware datetime for Eastern timezone
                            naive_dt = datetime.combine(current_date, datetime.min.time().replace(hour=hour, minute=minute))
                            slot_start = eastern.localize(naive_dt)
                            slot_end = slot_start + timedelta(minutes=duration)
                            
                            # Check if this slot conflicts with busy times
                            is_free = True
                            for busy in busy_times:
                                # Parse busy times (they come as UTC with 'Z' suffix)
                                busy_start_str = busy['start'].replace('Z', '+00:00')
                                busy_end_str = busy['end'].replace('Z', '+00:00')
                                busy_start = datetime.fromisoformat(busy_start_str)
                                busy_end = datetime.fromisoformat(busy_end_str)
                                
                                # Convert to Eastern timezone for comparison
                                busy_start = busy_start.astimezone(eastern)
                                busy_end = busy_end.astimezone(eastern)
                                
                                if (slot_start < busy_end and slot_end > busy_start):
                                    is_free = False
                                    break
                            
                            # Compare with current time (timezone-aware)
                            current_time = datetime.now(eastern)
                            if is_free and slot_start > earliest_booking_time:
                                available_slots.append({
                                    "date": current_date.strftime("%Y-%m-%d"),
                                    "time": slot_start.strftime("%H:%M"),
                                    "day": current_date.strftime("%A"),
                                    "formatted": slot_start.strftime("%A, %B %d at %I:%M %p EST")
                                })
                
                current_date += timedelta(days=1)
            
            # Limit to first 5 slots to avoid overwhelming the user
            available_slots = available_slots[:5]
            
            return {
                "available_slots": available_slots,
                "duration": duration,
                "timezone": "EST"
            }
            
        except HttpError as error:
            print(f"An error occurred: {error}")
            return {"error": f"Calendar check failed: {error}"}

    def create_google_meet_event(self, date: str, time: str, duration: int, 
                                attendee_email: str, topic: str = "Coffee Chat") -> Dict:
        """Create real Google Calendar event with Google Meet"""
        try:
            # Parse datetime
            event_datetime = datetime.strptime(f"{date} {time}", "%Y-%m-%d %H:%M")
            end_datetime = event_datetime + timedelta(minutes=duration)
            
            # Create event
            event = {
                'summary': f'Coffee Chat with Vach - {topic}',
                'description': f'Coffee chat discussion about: {topic}\n\nLooking forward to our conversation!',
                'start': {
                    'dateTime': event_datetime.isoformat(),
                    'timeZone': 'America/New_York',
                },
                'end': {
                    'dateTime': end_datetime.isoformat(),
                    'timeZone': 'America/New_York',
                },
                'attendees': [
                    {'email': attendee_email},
                ],
                'conferenceData': {
                    'createRequest': {
                        'requestId': f"meet_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
                        'conferenceSolutionKey': {'type': 'hangoutsMeet'}
                    }
                },
                'reminders': {
                    'useDefault': False,
                    'overrides': [
                        {'method': 'email', 'minutes': 60},
                        {'method': 'popup', 'minutes': 15},
                    ],
                },
            }
            
            # Create the event
            event_result = self.google_api.calendar_service.events().insert(
                calendarId='primary', 
                body=event,
                conferenceDataVersion=1,
                sendUpdates='all'
            ).execute()
            
            meet_link = None
            if 'conferenceData' in event_result and 'entryPoints' in event_result['conferenceData']:
                for entry_point in event_result['conferenceData']['entryPoints']:
                    if entry_point['entryPointType'] == 'video':
                        meet_link = entry_point['uri']
                        break
            
            print(f"📅 Created Google Calendar event:")
            print(f"   Event ID: {event_result['id']}")
            print(f"   Date: {date}")
            print(f"   Time: {time}")
            print(f"   Duration: {duration} minutes")
            print(f"   Attendee: {attendee_email}")
            print(f"   Topic: {topic}")
            print(f"   Meet Link: {meet_link}")
            
            return {
                "event_id": event_result['id'],
                "meet_link": meet_link,
                "calendar_link": event_result['htmlLink'],
                "date": date,
                "time": time,
                "duration": duration,
                "attendee": attendee_email,
                "topic": topic,
                "success": True
            }
            
        except HttpError as error:
            print(f"An error occurred: {error}")
            return {"error": f"Event creation failed: {error}", "success": False}

    def send_confirmation_email(self, attendee_email: str, event_details: str) -> Dict:
        """Send real confirmation email via Gmail API"""
        try:
            details = json.loads(event_details) if isinstance(event_details, str) else event_details
            
            # Format the time nicely
            event_time = datetime.strptime(f"{details['date']} {details['time']}", "%Y-%m-%d %H:%M")
            formatted_time = event_time.strftime("%A, %B %d at %I:%M %p EST")
            
            email_content = f"""Hi there!

This is an automated confirmation from Vach's booking system.

Your coffee chat with Vach is confirmed:

📅 When: {formatted_time}
📹 Google Meet: {details.get('meet_link', 'Link will be in calendar invite')}
💬 Topic: {details.get('topic', 'General chat')}
📧 Calendar Invite: Sent to {attendee_email}

I'm looking forward to our conversation!

Best regards,
Vach
"""
            
            # Create message
            message = MIMEText(email_content)
            message['to'] = attendee_email
            message['from'] = 'vachagan.melikian@rutgers.edu'  # Your email
            message['subject'] = f'Coffee Chat Confirmed - {formatted_time}'
            
            # Encode message
            raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
            
            # Send email
            send_result = self.google_api.gmail_service.users().messages().send(
                userId='me',
                body={'raw': raw_message}
            ).execute()
            
            print(f"📧 Confirmation email sent to: {attendee_email}")
            print(f"   Message ID: {send_result['id']}")
            
            return {
                "success": True,
                "email_sent_to": attendee_email,
                "message_id": send_result['id'],
                "timestamp": datetime.now().isoformat()
            }
            
        except HttpError as error:
            print(f"An error occurred: {error}")
            return {"error": f"Email sending failed: {error}", "success": False}

    def execute_tool(self, tool_call):
        """Execute the appropriate tool based on the tool call"""
        tool_name = tool_call.name
        parameters = tool_call.parameters
        
        if tool_name == "check_calendar_availability":
            return self.check_calendar_availability(**parameters)
        elif tool_name == "create_google_meet_event":
            return self.create_google_meet_event(**parameters)
        elif tool_name == "send_confirmation_email":
            return self.send_confirmation_email(**parameters)
        else:
            return {"error": f"Unknown tool: {tool_name}"}

    def chat(self, message: str, conversation_history: List = None):
        """Main chat interface with the agent"""
        if conversation_history is None:
            conversation_history = self.conversation_history

        preamble = """
        You are Vach's coffee chat booking assistant. You help people schedule 30-minute virtual coffee chats with Vach Melikian, a CS student at Rutgers who has experience in mobile development at Twitch and Fidelity.

        Your job is to:
        1. Understand what the person wants to discuss (career advice, technical questions, collaboration, etc.)
        2. Check Vach's real calendar availability using Google Calendar
        3. Help them pick a suitable time from available slots
        4. Create the Google Meet event in the calendar
        5. Send a professional confirmation email via Gmail

        Be friendly, professional, and helpful. Always ask for their email when you need to create the event.
        When showing available times, present them clearly and ask the user to pick one.
        """

        response = self.co.chat(
            model="command-r-08-2024",
            message=message,
            tools=self.tools,
            preamble=preamble,
            chat_history=conversation_history
        )

        # Handle tool calls
        while response.tool_calls:
            print(f"\n🤖 Agent wants to use tools: {[tc.name for tc in response.tool_calls]}")
            
            tool_results = []
            for tool_call in response.tool_calls:
                result = self.execute_tool(tool_call)
                tool_results.append({
                    "call": tool_call,
                    "outputs": [result]
                })

            # Continue conversation with tool results
            response = self.co.chat(
                model="command-r-08-2024",
                message="",
                tools=self.tools,
                chat_history=response.chat_history,
                tool_results=tool_results
            )

        # Update conversation history
        self.conversation_history = response.chat_history
        return response.text

# Setup instructions
print("🔧 Setup Instructions:")
print("1. Go to Google Cloud Console (console.cloud.google.com)")
print("2. Create a new project or select an existing one")
print("3. Enable the Calendar API and Gmail API")
print("4. Create OAuth 2.0 credentials (Desktop application)")
print("5. Download the credentials.json file to this directory")
print("6. Set your Cohere API key below")
print("7. Run the agent!")
print("=" * 60)

# Initialize the agent (you'll need to set your API key)
COHERE_API_KEY = "A7rMHHmxuMOmoDR4ZaG7G9wqyFBZ2b9CQ2N0sbPt"

try:
    agent = CoffeeChatAgent(COHERE_API_KEY)
    print("✅ Coffee Chat Booking Agent Ready with Real Google APIs!")
    print("=" * 60)

    # List available calendars first
    agent.list_available_calendars()
    print("=" * 60)
    
    # Test conversation
    def test_conversation(user_message):
        print(f"\n👤 User: {user_message}")
        print("-" * 40)
        response = agent.chat(user_message)
        print(f"🤖 Agent: {response}")
        print("=" * 60)

    # Example usage
    test_conversation("Hi, I'd like to book a coffee chat about iOS development")
    
except FileNotFoundError as e:
    print(f"❌ Setup Error: {e}")
    print("Please follow the setup instructions above.")

print("\n🚀 Features:")
print("• Real Google Calendar integration")
print("• Automatic Google Meet links")
print("• Professional Gmail confirmations")
print("• Multi-step agent reasoning")
print("• Natural language date parsing")

🔧 Setup Instructions:
1. Go to Google Cloud Console (console.cloud.google.com)
2. Create a new project or select an existing one
3. Enable the Calendar API and Gmail API
4. Create OAuth 2.0 credentials (Desktop application)
5. Download the credentials.json file to this directory
6. Set your Cohere API key below
7. Run the agent!
✅ Coffee Chat Booking Agent Ready with Real Google APIs!
Available calendars:
  - Personal (ID: vachik123@gmail.com)
    ^ This is your PRIMARY calendar
  - TELUS (ID: ctnh0906nf1petnebon2sfbbuc@group.calendar.google.com)
  - UCL Academic Calendar (ID: oqm83ho8s301ksvcr3o82n0nimmijjq4@import.calendar.google.com)
  - Rutgers University Photography Club (ID: ruphotographyclub@gmail.com)
  - NYT Astronomy and Space Calendar (ID: nytimes.com_89ai4ijpb733gt28rg21d2c2ek@group.calendar.google.com)
  - Holidays in United States (ID: en.usa#holiday@group.v.calendar.google.com)

👤 User: Hi, I'd like to book a coffee chat about iOS development
----------------------------

In [8]:
test_conversation("Monday at noon works for me. my email is vachagan.melikian@rutgers.edu")


👤 User: Monday at noon works for me. my email is vachagan.melikian@rutgers.edu
----------------------------------------

🤖 Agent wants to use tools: ['create_google_meet_event']
📅 Created Google Calendar event:
   Event ID: mg9837efpko0l99endv9qn2d8c
   Date: 2023-09-29
   Time: 12:00
   Duration: 30 minutes
   Attendee: vachagan.melikian@rutgers.edu
   Topic: iOS development coffee chat
   Meet Link: https://meet.google.com/drh-eqme-asy

🤖 Agent wants to use tools: ['send_confirmation_email']
📧 Confirmation email sent to: vachagan.melikian@rutgers.edu
   Message ID: 1998f84c97c8915e
🤖 Agent: Your Google Meet event has been successfully created for Monday, September 29 at 12:00 PM EST.

I have sent a confirmation email to vachagan.melikian@rutgers.edu with the event details. You should receive a confirmation shortly.

The event ID is mg9837efpko0l99endv9qn2d8c and the Google Meet link is https://meet.google.com/drh-eqme-asy.
